# Behavioural Driven Development Testing for Jupyter Notebooks

Handy way to process the run unit tests (via doctest) and integration tests (via behave) in jupyter notebooks (.ipynb) containing Python functions.
The script will convert an .ipynb to a string format (basically a .py file), loads them as modules, and runs the tests on them.
To run it in the console, do:

    python -m pytest --verbose --disable-warnings --nbval test_ipynb.ipynb

The script should tell you which ipynb file's doctests has failed (e.g. srgan_train.ipynb).
You can then open up this very jupyter notebook to debug and inspect the situation further.

In [1]:
from features.environment import _load_ipynb_modules
import behave.__main__

import doctest
import sys

def _doctest_ipynb(path: str):
    """
    Runs doctest on loaded modules from a .ipynb file
    """
    module = _load_ipynb_modules(ipynb_path=path)
    num_failures, num_attempted = doctest.testmod(m=module, verbose=True)
    if num_failures > 0:
        sys.exit(num_failures)

## Unit tests

Uses [doctest](https://en.wikipedia.org/wiki/Doctest).
Small tests for each individual function.

In [2]:
_doctest_ipynb('data_prep.ipynb')

Trying:
    download_to_path(path="highres/2017_Antarctica_Basler.csv",
                     url="https://data.cresis.ku.edu/data/rds/2017_Antarctica_Basler/csv_good/2017_Antarctica_Basler.csv")
Expecting:
    <Response [200]>
ok
Trying:
    check_sha256('highres/2017_Antarctica_Basler.csv')
Expecting:
    '53cef7a0d28ff92b30367514f27e888efbc32b1bda929981b371d2e00d4c671b'
ok
Trying:
    os.remove(path="highres/2017_Antarctica_Basler.csv")
Expecting nothing
ok
Trying:
    download_to_path(path="highres/2017_Antarctica_Basler.csv",
                     url="https://data.cresis.ku.edu/data/rds/2017_Antarctica_Basler/csv_good/2017_Antarctica_Basler.csv")
Expecting:
    <Response [200]>
ok
Trying:
    open('highres/2017_Antarctica_Basler.csv').readlines()
Expecting:
    ['LAT,LON,UTCTIMESOD,THICK,ELEVATION,FRAME,SURFACE,BOTTOM,QUALITY\n']
ok
Trying:
    os.remove(path="highres/2017_Antarctica_Basler.csv")
Expecting nothing
ok
3 items had no tests:
    data_prep
    data_prep.get_window_boun

In [3]:
_doctest_ipynb('srgan_train.ipynb')

Using TensorFlow backend.


Trying:
    metrics = {"generator_network": 'mse', "discriminator_network": 'accuracy'}
Expecting nothing
ok
Trying:
    models = compile_srgan_model(g_network=generator_network(), d_network=discriminator_network(), metrics=metrics)
Expecting nothing
ok
Trying:
    models['discriminator_model'].trainable
Expecting:
    True
ok
Trying:
    models['srgan_model'].get_layer(name='generator_network').trainable
Expecting:
    True
ok
Trying:
    models['srgan_model'].get_layer(name='discriminator_network').trainable
Expecting:
    False
ok
Trying:
    models['srgan_model'].count_params()
Expecting:
    8571362
ok
Trying:
    discriminator_network().input_shape
Expecting:
    (None, 32, 32, 1)
ok
Trying:
    discriminator_network().output_shape
Expecting:
    (None, 1)
ok
Trying:
    discriminator_network().count_params()
Expecting:
    6828033
ok
Trying:
    generator_network().input_shape
Expecting:
    [(None, 8, 8, 1), (None, 40, 40, 1), (None, 16, 16, 1)]
ok
Trying:
    generator_network

## Integration tests

Uses [behave](https://github.com/behave/behave).
Medium sized tests which checks that components work together properly.
Ensures that the behaviour of features (made up of units) is sound.

In [4]:
behave.__main__.main("features/data_prep.feature")

@fixture.data_prep
Feature: Data preparation # features/data_prep.feature:3
  In order to have reproducible data inputs for everyone
  As a data scientist,
  We want to share cryptographically secured pieces of the datasets
  Scenario Outline: Download and check data -- @1.1 Files to download and check                                                                # features/data_prep.feature:15
    Given this https://data.cresis.ku.edu/data/rds/2017_Antarctica_Basler/csv_good/2017_Antarctica_Basler.csv link to a file hosted on the web # features/steps/data_prep.py:5
    When we download it to highres/2017_Antarctica_Basler.csv                                                                                  # features/steps/data_prep.py:10
    Then the local file should have this 53cef7a0d28ff92b30367514f27e888efbc32b1bda929981b371d2e00d4c671b checksum                             # features/steps/data_prep.py:16

1 feature passed, 0 failed, 0 skipped
1 scenario passed, 0 failed, 0 skip

0